In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, lit
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

df = spark.read.option("header", "true") \
    .csv("/home/iceberg/data/events.csv") \
    .withColumn("event_date", expr("DATE_TRUNC('day', event_time)"))
    
df.join(df, lit(1) == lit(1)).take(5) # collect() is bad practice. Use show or take if not aggregated and sure that it is small

24/12/03 19:12:04 WARN Column: Constructing trivially true equals predicate, '1 = 1'. Perhaps you need to use aliases.


[Row(user_id='1037710827', device_id='532630305', referrer=None, host='www.zachwilson.tech', url='/', event_time='2021-03-08 17:27:24.241000', event_date=datetime.datetime(2021, 3, 8, 0, 0), user_id='1037710827', device_id='532630305', referrer=None, host='www.zachwilson.tech', url='/', event_time='2021-03-08 17:27:24.241000', event_date=datetime.datetime(2021, 3, 8, 0, 0)),
 Row(user_id='1037710827', device_id='532630305', referrer=None, host='www.zachwilson.tech', url='/', event_time='2021-03-08 17:27:24.241000', event_date=datetime.datetime(2021, 3, 8, 0, 0), user_id='925588856', device_id='532630305', referrer=None, host='www.eczachly.com', url='/', event_time='2021-05-10 11:26:21.247000', event_date=datetime.datetime(2021, 5, 10, 0, 0)),
 Row(user_id='1037710827', device_id='532630305', referrer=None, host='www.zachwilson.tech', url='/', event_time='2021-03-08 17:27:24.241000', event_date=datetime.datetime(2021, 3, 8, 0, 0), user_id='-1180485268', device_id='532630305', referrer=N

In [11]:
#sorted = df.repartition(10, col("event_date")) \
#        .sortWithinPartitions(col("event_date"), col("host")) \
#        .withColumn("event_time", col("event_time").cast("timestamp"))  # local sort in partition fast

#sortedTwo = df.repartition(10, col("event_date")) \
#        .sort(col("event_date"), col("host")) \
#        .withColumn("event_time", col("event_time").cast("timestamp"))  # global sort slow , avoid if possible
sorted = df.sortWithinPartitions(col("event_date"), col("host")) \
        .withColumn("event_time", col("event_time").cast("timestamp"))  # local sort in partition fast

sortedTwo = df.sort(col("event_date"), col("host")) \
        .withColumn("event_time", col("event_time").cast("timestamp"))  # global sort slow , avoid if possible

#sorted.show()
#sortedTwo.show()
#sorted.explain()
#sortedTwo.explain()

In [12]:
%%sql

CREATE DATABASE IF NOT EXISTS bootcamp

++
||
++
++

In [13]:
%%sql

DROP TABLE IF EXISTS bootcamp.events

++
||
++
++

In [14]:
%%sql

CREATE TABLE IF NOT EXISTS bootcamp.events (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (event_date);


++
||
++
++

In [15]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_sorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (event_date);

++
||
++
++

In [113]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_unsorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg;

++
||
++
++

In [17]:

start_df = df.repartition(4, col("event_date")).withColumn("event_time", col("event_time").cast("timestamp")) \
    

first_sort_df = start_df.sortWithinPartitions(col("event_date"), col("host"))

sorted = df.repartition(10, col("event_date")) \
        .sortWithinPartitions(col("event_date")) \
        .withColumn("event_time", col("event_time").cast("timestamp")) \

start_df.write.mode("overwrite").saveAsTable("bootcamp.events_unsorted")
first_sort_df.write.mode("overwrite").saveAsTable("bootcamp.events_sorted")

In [19]:
%%sql

SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'sorted' 
FROM demo.bootcamp.events_sorted.files

UNION ALL
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'unsorted' 
FROM demo.bootcamp.events_unsorted.files





size,num_files,sorted
4955506,4,sorted
5050842,4,unsorted


In [90]:
%%sql
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files FROM demo.bootcamp.events.files;

size,num_files
3145713,5


In [ ]:
%%sql 
SELECT COUNT(1) FROM bootcamp.matches_bucketed.files

count(1)
3665
